In [18]:
import en_core_web_md
from process import parse_sentence
from tqdm import tqdm
from transformers import AutoTokenizer, BertModel, GPT2Model
from mapper import Map, deduplication

import torch
import numpy as np
from utils import compress_attention, create_mapping, BFS, build_graph, is_word
from multiprocessing import Pool
import spacy
import en_core_web_md
import torch
from transformers import AutoTokenizer, BertModel, GPT2Model
from constant import invalid_relations_set
from utils import compress_attention,build_graph
import re


In [19]:
language_model = 'allenai/scibert_scivocab_uncased'
tokenizer = AutoTokenizer.from_pretrained(language_model)
encoder = BertModel.from_pretrained(language_model)
encoder.eval()
use_cuda =False
threshold = 0.01
from spacy.matcher import PhraseMatcher

# def process_matrix(attentions, layer_idx = -1, head_num = 0, avg_head=False, trim=True, use_cuda=True):
#     if avg_head:
#         if use_cuda:
#             attn =  torch.mean(attentions[0][layer_idx], 0).cpu()
#         else:
#             attn = torch.mean(attentions[0][layer_idx], 0)
#         attention_matrix = attn.detach().numpy()
#     else:
#         attn = attentions[0][layer_idx][head_num]
#         if use_cuda:
#             attn = attn.cpu()
#         attention_matrix = attn.detach().numpy()

#     attention_matrix = attention_matrix[1:-1, 1:-1]

#     return attention_matrix


###test ###



nlp = spacy.load("en_core_web_md")
matcher1 = PhraseMatcher(nlp.vocab,attr='POS')

terms = ['relate to','positioned through','engageable with','so as to','makes it possible to']
patterns=[nlp(term) for term in terms]
matcher1.add('VerbPhrase',patterns)



###test ###


with open ('examples/patent_example.txt','r') as f:
    num_file = sum([1 for i in open('examples/patent_example.txt', "r")])
    #tqdm summmarization
    for idx, line in enumerate(tqdm (f,total=num_file)):

        ##modification starts below###
#         sentence= line.strip()
        sentence = re.sub(' +', ' ',line.strip())
        
        ##modification ends here\####
#         if len(sentence):
#             valid_triplets = []
#             for sent in nlp(sentence).sents:
#                 sentence= sent.text
#                 tokenizer_name = str(tokenizer.__str__)
#                 inputs, tokenid2word_mapping, token2id, noun_chunks  = create_mapping(sentence, return_pt=True, nlp=nlp, tokenizer=tokenizer)

#                 with torch.no_grad():
#                     if use_cuda:
#                         for key in inputs.keys():
#                             inputs[key] = inputs[key].cuda()
#                     outputs = encoder(**inputs, output_attentions=True)
#                 trim = True
#                 if 'GPT2' in tokenizer_name:
#                     trim  = False

#                 '''
#                 Use average of last layer attention : page 6, section 3.1.2
#                 '''
#                 attention = process_matrix(outputs[2], avg_head=True, trim=trim, use_cuda=use_cuda)

#                 merged_attention = compress_attention(attention, tokenid2word_mapping)
#                 attn_graph = build_graph(merged_attention)
#             print ('attention:{}'.format(attention))
#             print ('merged_attention:{}'.format(merged_attention))
#             print ("attn_graph:{}".format(attn_graph))
#             print ("attention_shape:{0},merged_attention_shape:{1},attention_graph_shape:{2}".\
#                    format(attention.shape,              
#                           merged_attention.shape,
#                          len(attn_graph)))

        valid_triplets = []
        count = 0
        doc = nlp(sentence)
        print ("count:{}".format(count))
        
        ###test for verb phrases start###
        matches = matcher1(doc)
        verb_phrase_start_chunk = []
        verb_phrase_end_chunk = []
        verb_phrase_chunk=[]
        for match_id, start, end in matches:
            span = doc[start:end]
            verb_phrase_start_chunk.append(start)
            verb_phrase_end_chunk.append(end)
            verb_phrase_chunk.append(span.text)
            # print("start: %s, end: %s text :%s" % (start,end,span.text))
            
        ### test for verb phrase ends####
        
        
        
        for sent in nlp(sentence).sents:
            print ("sent:{}".format(sent))
            print ("sent start :%s sent end :%s"% (sent.start, sent.end))
            
#         start_chunk = []
#         end_chunk = []
#         noun_chunks = []

        noun_start_chunk = []
        noun_end_chunk = []
        noun_chunks = []

        for chunk in doc.noun_chunks:
            noun_chunks.append(chunk.text)
            start_chunk.append(chunk.start)
            end_chunk.append(chunk.end)
            
#         start_chunk= sorted(start_chunk + verb_phrase_start_chunk)
#         end_chunk = sorted (end_chunk +verb_phrase_end_chunk)
        chunk_id = 0
        token2id ={ }
        mode = 1
        sentence_mapping = []

        #### modification starts below,
        ### token2id [sentence_mapping[-1]]= len(token2id) --->token2id [sentence_mapping[-1]]= len(sentence_mapping)-1
        ### because token2id is a dictionary, when you len it, it will automatically remove the duplicated keys, this will
        ### cause problems when the words occurs more than once

            
        dic = dict (zip (start_chunk+verb_phrase_start_chunk,noun_chunks+verb_phrase_chunk))
        print (dic)
        dic = {k: v for k, v in sorted(dic.items(), key=lambda dic:dic[0],reverse= False)}

 
        start_chunk= list(dic.keys())
        chunks = list(dic.values())
        end_chunk = sorted(end_chunk+ verb_phrase_end_chunk)
        print ('start_chunk: %s, end_chunk: %s'%(str(start_chunk),str(end_chunk)))
        print (dic)
        for idx,token in enumerate (doc):
            if idx in start_chunk:
                

                mode= 1
                sentence_mapping.append (chunks[chunk_id])
                token2id [sentence_mapping[-1]]= len(sentence_mapping)-1
#                 token2id [sentence_mapping[-1]]= len(token2id)
                chunk_id +=1 
                continue
            if idx in end_chunk:
                mode = 0

                
            if mode == 0:
                sentence_mapping.append (token.text)
                token2id [sentence_mapping[-1]]= len(sentence_mapping)-1
#                 token2id [sentence_mapping[-1]]= len(token2id)

        ### modication ends here 
                
        print ('sentence_mapping:{}'.format (sentence_mapping))
        
        token_ids = []
        tokenid2word_mapping = []
        count = 0
        for token in sentence_mapping:
            subtoken_ids = tokenizer(str(token), add_special_tokens=True)['input_ids']
            tokenid2word_mapping += [ token2id[token] ]*len(subtoken_ids)  # have a look at the following sample result and you will know.
            token_ids += subtoken_ids
#             print ("token_ids:{}".format(token_ids))
#             print ("tokenid2word_mapping:{}".format(tokenid2word_mapping))
#             print ("subtoken_ids:{}".format (subtoken_ids))
#             print ("----next round----")
#             count +=1
#             if count ==3 :
#                 break

                
        print ("noun_chunks:{}".format(noun_chunks))
        print ("start_chunk:{}".format(start_chunk))
        print ("end_chunk:{}".format(end_chunk))
        print ("token2id:{}".format(token2id))
        print ("token_ids:{}".format(token_ids))
        print ("tokenid2word_mapping:{}".format(tokenid2word_mapping))
        print ("subtoken_ids:{}".format (subtoken_ids))
        tokenizer_name = str(tokenizer.__str__)

        outputs = {
            'input_ids': [tokenizer.cls_token_id] + token_ids + [tokenizer.sep_token_id],
            'attention_mask': [1]*(len(token_ids)+2),
            'token_type_ids': [0]*(len(token_ids)+2)     }

        for key, value in outputs.items():
            outputs[key] = torch.from_numpy(np.array(value)).long().unsqueeze(0)
        print("outputs:{}".format(outputs))
        print ("len(tokenids):{}".format(len(token_ids))) #equals to the length of original attn matrix len# 
        use_cuda = False
        with torch.no_grad():
            if use_cuda:
                for key in outputs.keys():
                    outputs[key] = outputs[key].cuda()
            outputs_encoder = encoder(**outputs, output_attentions=True)
#         print ("outputs_encoder:{}".format(outputs_encoder))
#         print ("outputs_encoder[2]:{}".format(outputs_encoder[2]))        
        trim =True
        attn = outputs_encoder[2][0][-1]
        print ("attn:{}".format(attn))
        attn =torch.mean(attn, 0)
        attention_matrix = attn.detach().numpy()
        attention_matrix = attention_matrix[1:-1, 1:-1]

#         print ("attn:{}".format(attn))    
#         print ('attention_matrix:{}'.format(attention_matrix))
        print (attention_matrix.shape)
        merged_attention = compress_attention(attention_matrix, tokenid2word_mapping)
        print ("merged_attention:{},shape:{}".format (merged_attention,merged_attention.shape))
        attn_graph = build_graph(merged_attention)
        print ("attn_graph:{}".format(attn_graph))
        tail_head_pairs=[]
        
        
        print ("len(tokenids):{}".format(len(token_ids))) #equals to the length of original attn matrix len# 
        print ("len(tokentokenid2word_mapping):{}".format(len(tokenid2word_mapping))     )   
        print ("attention_matrix:{}".format(attention_matrix.shape))
        print ("merged_attention shape:{}".format (merged_attention.shape))
        id2token = {key:value for key,value in enumerate(sentence_mapping)}
#############modification starts here ################
        
        i=0
        id2noun_chunk_token=dict()
        for key,value in  id2token.items():
            if value in noun_chunks:
                id2noun_chunk_token[i]=key
                i=i+1
                
#############modification ends here ###########################
    #############
        def head_tail_pairs_append(head_tail_pairs,sent,id2noun_chunk_token,start_chunk,token2id):
#             print ('sent.start:%s, sent.end:%s'%(sent.start,sent.end))
            for i1,start1 in enumerate(start_chunk):
#                 print ('i1:%s, start1 :%s' %(i1, start1))
                if start1 < sent.start:
                    continue
                if start1 >=sent.end:
                    break
                for i2,start2 in enumerate(start_chunk):
        #             print ("i2:%s, start2 :%s"%(i2, start2))
                    if start2<start1 :
                        continue
                    if start2>= sent.end: 
                        break
                    if start1!=start2:
        #                 print ("succeed, start1 :%s,start2:%s"%(noun_chunks[i1],noun_chunks[i2]))
#                         print ('sc')
                        head_tail_pairs.append((id2noun_chunk_token[i1],id2noun_chunk_token[i2]))
            return head_tail_pairs
        for sent in nlp(sentence).sents:
            tail_head_pairs = head_tail_pairs_append(tail_head_pairs,sent,id2noun_chunk_token,start_chunk,token2id)
            
            
            #######################################       

#should starts from here
# add'same sentence ' limit here
# first, the pair is limited in the same sentence, 2nd, the funciton is preventing meaningless iteration beyond same sentence.         
     
#         for sent in nlp(sentence).sents:
#             for (head,start_number) in zip(noun_chunks,start_chunk):
#                 if start_number < sent.end_number:
#                     for (tail,end_number) in zip(noun_chunks,end_chunk):
#                         if end_number > sent.end_number:
#                             continue
#                         if head != tail:
#                             tail_head_pairs.append((token2id[head], token2id[tail]))

##should ends here
                    

    
    
    

        black_list_relation = set([ token2id[n]  for n in noun_chunks ])



        
        all_relation_pairs = []
#         id2token = { value: key for key, value in token2id.items()}
        #### modification starts below,
        ### id2token = { value: key for key, value in token2id.items()} --->id2token = {key:value for key,value in enumerate(sentence_mapping)}
        ### because in token2id dictionary, there is no duplicate tokens, so it won't be reflect the positions with these tokens
        ### but the later algorithm needs all positions 
        id2token = {key:value for key,value in enumerate(sentence_mapping)}
        ### modication ends here 
        print ("tail_head_pairs:{}".format(tail_head_pairs)) 
        print ("id2token:{}".format(id2token))
        print ("len_sentence_mapping:{}".format(len(sentence_mapping)))
# '''
#         the following is a sample of the result
#         noun_chunks:['A highly fire-resistant and environmentally-friendly panel', '2 mm to 28 mm', 'a blending', 'magnesium compounds',
#         'sodium silicate', 
#         'kaolin', 'fillers', 'additives', 'the core materials', '4 layers', 'fire-resistant glass fiber meshes', 'fabrics', 
#         'a proprietary ITC process', 'the chemical reactions', 'the ingredients', 'sufficient heat', 'external supply', 'energy',
#         'the panels', '24 hours', '10 days', 'The use', 'waste materials', 'energy-saving curing system', 'no gas emission manufacturing process', 
#         'this panel', "the world's highest-rated fire resistance", '5 hours', 'high flexural strength', 'low density', 'durability', 
#         'effective water-resistance']
#         start_chunk:[0, 11, 20, 23, 26, 29, 31, 34, 37, 43, 46, 53, 56, 62, 66, 70, 73, 76, 78, 85, 89, 92, 95, 98, 104, 112, 121, 130, 133, 137, 140, 142]
# end_chunk:[10, 16, 22, 25, 28, 30, 32, 35, 40, 45, 52, 54, 60, 65, 68, 72, 75, 77, 80, 87, 91, 94, 97, 103, 109, 114, 129, 132, 136, 139, 141, 146]

#         end_chunk:[10, 16, 22, 25, 28, 30, 32, 35, 40, 45, 52, 54, 60, 65, 68, 72, 75, 77, 80, 87, 91, 94, 97, 103, 109, 114, 129, 132, 136, 139, 141, 146]
#         token2id:{'A highly fire-resistant and environmentally-friendly panel': 0, 'of': 56, '2 mm to 28 mm': 2, 'may': 35, 'be': 35, 'manufactured': 5, 
# 'by': 19, 'a blending': 7, 'magnesium compounds': 8, ',': 59, 'sodium silicate': 10, 'kaolin': 11, 'fillers': 12, 'and': 60, 'additives': 14, 'to': 46, 
# 'form': 16, 'the core materials': 17, 'reinforced': 18, '4 layers': 19, 'fire-resistant glass fiber meshes': 20, 'fabrics': 21, '.': 61, 'Using': 23,
# 'a proprietary ITC process': 24, 'that': 25, 'accelerates': 26, 'the chemical reactions': 27, 'the ingredients': 28, 'generate': 29, 'sufficient heat': 30, 
# 'without': 31, 'external supply': 32, 'energy': 33, 'the panels': 34, 'completely': 35, 'cured': 36, 'within': 37, '24 hours': 38, 'instead': 39, '10 days': 40,
# 'The use': 41, 'waste materials': 42, 'energy-saving curing system': 43, 'no gas emission manufacturing process': 44, 'combined': 45, 'make': 46, 'this panel':
# 47, 'an': 48, 'eco': 49, '-': 50, 'friendly': 51, 'product': 52, 'which': 53, 'offers': 54, "the world's highest-rated fire resistance": 55, '5 hours': 56,
# 'high flexural strength': 57, 'low density': 58, 'durability': 59, 'effective water-resistance': 60}
#         token_ids:[138, 3023, 1783, 118, 13676, 1105, 4801, 1193, 118, 4931, 5962, 1104, 123, 2608, 1106, 1743, 2608]
#         tokenid2word_mapping:[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 56, 2, 2, 2, 2, 2]
#         subtoken_ids:[123, 2608, 1106, 1743, 2608]
#         '''

        
        #####

#             for triplets in parse_sentence(sent.text,tokenizer,encoder,nlp,use_cuda=False):
#                 print (triplets)
#                 valid_triplets.append (triplets)
#                 if len(valid_triplets) > 0:
#                     # Map
#                     mapped_triplets = []
#                 for triplet in valid_triplets:
#                     head = triplet['h']
#                     tail = triplet['t']
#                     relations = triplet['r']
#                     conf = triplet['c']     
#                     if conf < threshold:
#                         continue
#                     mapped_triplet = Map(head, relations, tail)
#                     if 'h' in mapped_triplet:
#                         mapped_triplet['c'] = conf
#                         mapped_triplets.append(mapped_triplet)
#                 output = { 'line': idx, 'tri': deduplication(mapped_triplets) }
#                 print (output)
#                 if include_sentence:
#                     output['sent'] = sentence
#                 if len(output['tri']) > 0:
#                     g.write(json.dumps( output )+'\n')
        break
        ### inputs: A highly fire-resistant and environmentally-friendly panel of 2 mm to 28 mm may be manufactured by a 
        ##blending of magnesium compounds, sodium silicate, kaolin, fillers, and additives to form the core materials, reinforced 
        ###by 4 layers of fire-resistant ....
        ### outputs: 
        ### 1, outputs:
        ### 2, token2idword_mapping:[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 56, 
        ### 2, 2, 2, 2, 2, 35, 35, 5, 19, 7, 7, 56, 8, 8, 8, 59, 10, 10, 10, 10, 59, 11, 11, 59, 12, 12, 59, 60, 14, 14, 14, 46, 16, 17, 17, 17, 59, 18, 19, 19, 19, 56, 20, 20, 20, 
        ### 20, 20, 20, 20, 60, 21, 21, 61, 23, 24, 24, 24, 24, 24, 25, 26, 26, 27, 27, 27, 56, 28, 28, 46, 29, 30, 30, 31, 32, 32, 56, 33, 59, 34, 34, 35, 35, 35, 36, 37, 38, 38, 
        ### 39, 56, 40, 40, 61, 41, 41, 56, 42, 42, 59, 43, 43, 43, 43, 43, 43, 60, 44, 44, 44, 44, 44, 45, 46, 46, 47, 47, 48, 49, 49, 50, 51, 52, 53, 54, 55, 55, 55, 55, 55, 55,
        ### 55, 55, 55, 56, 56, 56, 59, 57, 57, 57, 57, 57, 59, 58, 58, 59, 59, 59, 59, 60, 60, 60, 60, 60, 61]

  0%|          | 0/32 [00:00<?, ?it/s]

count:0
sent:The invention relates to a roadable aircraft vehicle and related systems.
sent start :0 sent end :12
sent:An example roadable aircraft vehicle includes a vehicle drive system including an engine and gearbox selectively engageable with an automotive driveline and at least one propeller , a user interface including a display for controlling the drive system in an automotive mode including a steering wheel and in a flight mode including a control stick , a control system for switching between the flight mode and the automotive mode, and a system for locking the propeller during the automotive mode.
sent start :12 sent end :94
sent:The invention also relates to aircraft systems and elements such as an airfoil having a nominal profile, a folding wing , and an occupant crash protection system for an aircraft
sent start :94 sent end :125
{0: 'The invention', 2: 'a roadable aircraft vehicle', 4: 'related systems', 9: 'An example roadable aircraft vehicle', 12: 'a vehicle drive sys

  0%|          | 0/32 [00:00<?, ?it/s]

attn:tensor([[[1.3194e-03, 5.2866e-03, 4.8729e-02,  ..., 2.4504e-03,
          1.0390e-03, 1.0441e-03],
         [6.0316e-06, 9.1629e-04, 2.4702e-02,  ..., 7.0470e-03,
          8.1012e-03, 7.8630e-03],
         [1.1550e-03, 6.8930e-03, 4.8454e-03,  ..., 1.6150e-03,
          5.7780e-03, 5.8672e-03],
         ...,
         [8.2698e-03, 5.2186e-03, 1.1070e-03,  ..., 3.2867e-03,
          4.8564e-03, 4.7310e-03],
         [2.7200e-02, 1.3011e-02, 2.6921e-03,  ..., 5.0922e-04,
          2.0170e-03, 1.9465e-03],
         [2.5338e-02, 1.2095e-02, 2.6254e-03,  ..., 5.5666e-04,
          2.3277e-03, 2.2457e-03]],

        [[3.8701e-04, 1.2004e-03, 2.6828e-05,  ..., 2.3535e-05,
          1.3305e-02, 1.0456e-02],
         [3.7791e-04, 3.7617e-02, 1.6443e-04,  ..., 7.0442e-06,
          6.1450e-05, 5.2941e-05],
         [1.6356e-03, 1.1078e-02, 2.1066e-03,  ..., 6.9912e-04,
          2.6600e-03, 2.5690e-03],
         ...,
         [1.0210e-03, 6.4660e-04, 7.3620e-04,  ..., 2.1362e-03,
          

KeyError: 30

In [6]:
from multiprocessing import Pool
from utils import BFS
def bfs(args):
    s, end, graph, max_size, black_list_relation = args
    return BFS(s, end, graph, max_size, black_list_relation)
def global_initializer(nlp_object):
    global spacy_nlp
    spacy_nlp = nlp_object
def check_relations_validity(relations):
    for rel in relations:
        if rel.lower() in invalid_relations_set or rel.isnumeric():
            return False
    return True
def filter_relation_sets(params):
    triplet, id2token = params

    triplet_idx = triplet[0]
    confidence = triplet[1]
    head, tail = triplet_idx[0], triplet_idx[-1]
    if head in id2token and tail in id2token:
        head = id2token[head]
        tail = id2token[tail]
        relations = [ spacy_nlp(id2token[idx])[0].lemma_  for idx in triplet_idx[1:-1] if idx in id2token ]
        if len(relations) > 0 and check_relations_validity(relations) and head.lower() not in invalid_relations_set and tail.lower() not in invalid_relations_set:
            return {'h': head, 't': tail, 'r': relations, 'c': confidence }
    return {}


## candidate_facts_filter### end
print (tail_head_pairs[0])
for pair in tail_head_pairs:    
    print (pair)
    candidate_facts = BFS(pair[0],pair[1],attn_graph,max(tokenid2word_mapping),black_list_relation)







# with Pool(10) as pool:
#     params = [  ( pair[0], pair[1], attn_graph, max(tokenid2word_mapping), black_list_relation, ) for pair in tail_head_pairs]

#     for output in pool.imap_unordered(bfs, params):

#         if len(output):
#             all_relation_pairs += [(output[0], id2token)]
## each relation pair include a pair of relation (h,r,t) and the id2token dictionary##
## id2token is added because 'filter_relation_sets' need it as the input to assure the word is in token list 

# print ('all_r_p',all_relation_pairs[0:4])

# triplet_text = []
# with Pool(10, global_initializer, (nlp,)) as pool:
#     for triplet in pool.imap_unordered(filter_relation_sets, all_relation_pairs):
#         if len(triplet) > 0:
#             triplet_text.append(triplet)




#             '''
#             output[0], [([0, 1, 2], 0.0370228560641408)]
# all_relation_pairs[0] 
# [(([0, 1, 2], 0.0370228560641408), {0: 'A highly fire-resistant and environmentally-friendly panel',
# 76: 'of', 2: '2 mm to 28 mm', 47: 'may', 48: 'be', 5: 'manufactured', 24: 'by', 7: 'a blending', 9: 'magnesium compounds', 82: ',', 
# 11: 'sodium silicate', 13: 'kaolin', 15: 'fillers', 84: 'and', 18: 'additives', 65: 'to', 20: 'form', 21: 'the core materials', 23: 
# 'reinforced', 25: '4 layers', 27: 'fire-resistant glass fiber meshes', 29: 'fabrics', 86: '.', 31: 'Using', 32: 'a proprietary ITC process',
# 33: 'that', 34: 'accelerates', 35: 'the chemical reactions', 37: 'the ingredients', 39: 'generate', 40: 'sufficient heat', 41: 'without', 
# 42: 'external supply', 44: 'energy', 46: 'the panels', 49: 'completely', 50: 'cured', 51: 'within', 52: '24 hours', 53: 'instead', 
# 55: '10 days', 57: 'The use', 59: 'waste materials', 61: 'energy-saving curing system', 63: 'no gas emission manufacturing process',
# 64: 'combined', 66: 'make', 67: 'this panel', 68: 'an', 69: 'eco', 70: '-', 71: 'friendly', 72: 'product', 73: 'which', 74: 'offers', 
# 75: "the world's highest-rated fire resistance", 77: '5 hours', 79: 'high flexural strength', 81: 'low density', 83: 'durability', 
# 85: 'effective water-resistance'})]
#             '''

(0, 3)
(0, 3)


TypeError: 'int' object is not subscriptable

In [ ]:
pass_s=tail_head_pairs[1][0]
pass_end=tail_head_pairs[1][1]
pass_parm_attn_graph=attn_graph
pass_parm_max=max(tokenid2word_mapping)
pass_parm_black_list_relation=black_list_relation


In [ ]:
def check_single_relation_validity(relations):
    if relations.lower() in invalid_relations_set or relations.isnumeric():
        return False
    else:
        return True

def candidate_facts_relation_filter(candidate_facts):
    filtered_candidate_facts=[]
    for candidate_fact in candidate_facts:
        head=candidate_fact[0][0]
        tail=candidate_fact[0][2]
        relations=candidate_fact[0][1]
#         print ('%s %s %s'%(id2token[head],id2token[tail],id2token[relations]))
#         print ( len(id2token[relations]) > 0 )
#         print (id2token[relations])
#         print (check_single_relation_validity(id2token[relations]) )
#         print (id2token[head].lower() not in invalid_relations_set )
#         print (id2token[tail].lower() not in invalid_relations_set)
        if len(id2token[relations]) > 0 and check_single_relation_validity(id2token[relations]) and id2token[head].lower() not in invalid_relations_set and id2token[tail].lower() not in invalid_relations_set:
            filtered_candidate_facts.append(candidate_fact)

    print (filtered_candidate_facts)
    if len(filtered_candidate_facts)>0:
        filtered_candidate_facts = sorted(filtered_candidate_facts, key=lambda x: x[1], reverse=True)

        filtered_candidate_facts = filtered_candidate_facts[0]
        return (filtered_candidate_facts)
    if len(filtered_candidate_facts)==0:
        return []

In [ ]:

# print (attn_graph)
from utils import BFS
# result =BFS(pass_s,pass_end,pass_parm_attn_graph,pass_parm_max,pass_parm_black_list_relation)
result= [([0, 1, 5], 0.044546898920089006)]
candidate_facts = candidate_facts_relation_filter(result)
print (candidate_facts)


print ("id2token:{}".format(id2token))

In [ ]:



def global_initializer(nlp_object):
    global spacy_nlp
    spacy_nlp = nlp_object
def filter_relation_sets(params):
    triplet, id2token = params
    global spacy
    spacy_nlp= en_core_web_md.load()
    triplet_idx = triplet[0]
    confidence = triplet[1]
    head, tail = triplet_idx[0], triplet_idx[-1]
    if head in id2token and tail in id2token:
        head = id2token[head]
        tail = id2token[tail]
        relations = [ spacy_nlp(id2token[idx])[0].lemma_  for idx in triplet_idx[1:-1] if idx in id2token ]
        if len(relations) > 0 and check_relations_validity(relations) and head.lower() not in invalid_relations_set and tail.lower() not in invalid_relations_set:
            return {'h': head, 't': tail, 'r': relations, 'c': confidence }
    return {}
all_relation_pairs= [ (perresult, id2token) for perresult in result]
from mapper import deduplication
triplet_text = []
global_initializer(nlp)
triplet_text = []
global spacy_nlp
f_r= [filter_relation_sets(result)for result in all_relation_pairs]



In [ ]:
print(f_r)

In [2]:
import en_core_web_md
nlp = en_core_web_md.load()

relations = nlp('relates to')[0].lemma_
print (relations)

relate


In [9]:
relations = nlp('relates to')
print (relations)

relates to


TypeError: Argument 'string' has incorrect type (expected str, got spacy.tokens.doc.Doc)